In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama

In [ ]:
llm = ChatOllama(model= 'llama3.2:1b')

In [ ]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [ ]:
def joke_node(state: JokeState):
    topic = state['topic']

    prompt = f""" you are a funny person and likes to joke about the given topic. Make a joke about the following topic: {topic}"""

    joke = llm.invoke(prompt).content

    return {'joke': joke}

def explanation_node(state: JokeState):

    joke = state['joke']

    prompt = f"""You are a boring person and likes to explain everything, even jokes! Explain the following joke: {joke}"""

    explanation = llm.invoke(prompt).content

    return {'explanation': explanation}

In [ ]:
graph = StateGraph(JokeState)

graph.add_node('joke_node', joke_node)
graph.add_node('explanation_node', explanation_node)

graph.add_edge(START, 'joke_node')
graph.add_edge('joke_node', 'explanation_node')
graph.add_edge('explanation_node', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [ ]:
workflow

In [ ]:
config1 = {'configurable': {'thread_id': '1'}}

initial_state = {'topic': 'Indian Politics'}

workflow.invoke(initial_state, config=config1)

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}

initial_state = {"topic": "Indians"}

workflow.invoke(initial_state, config=config2)

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({"topic": "pizza"}, config=config1)

In [ ]:
workflow.get_state(config1)

In [ ]:
workflow.get_state_history(config1)

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({"topic": "pasta"}, config=config2)

## Time Travel

In [ ]:
workflow.get_state(
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f078c8d-511f-60fb-8005-1d948efec61e",
        }
    }
)

In [ ]:
workflow.invoke(
    None,
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f078c8d-511f-60fb-8005-1d948efec61e",
        }
    },
)

In [ ]:
list(workflow.get_state_history(config1))

In [ ]:
workflow.update_state(
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f078c8d-511f-60fb-8005-1d948efec61e",
            "checkpoint_ns": "",
        }
    },
    {"topic": "samosa"},
)

In [ ]:
list(workflow.get_state_history(config1))

In [ ]:
workflow.invoke(
    None,
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f078c8d-511f-60fb-8005-1d948efec61e",
        }
    },
)

In [ ]:
list(workflow.get_state_history(config1))

## Fault Tolerance

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [ ]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [ ]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}


def step_2(state: CrashState) -> CrashState:
    print(
        "⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)"
    )
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}


def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [ ]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": "thread-1"}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": "thread-1"}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": "thread-1"}}))